Inspired by Gabyjuniors post, I looked into the exact cover approach for the Mondriaan puzzle. This allows for an exact solver, though at considerable cost for processing time.

- Every coordinate of the square is a point. 
- The posistions of a rectangle is a family of covers of these points.
- Add the constraint that each rectangle has an additional ID node to avoid using the same rectangle twice.

I used the Algorithm X approach as I have not used it before. 

An alternative would be to use a linear programming solver. This would be better for performance as there exist highly optimized libraries for these.

In [1]:
# Knuth's Algorithm X (dancing links) for solving an exact cover problem

def solve(X, Y, solution=[]):
    """
    A variation of the dancing links alogrithm for the exact cover problem.
    X is a dict containing sets of objects contained per node
    Y is a dict containing a list of all nodes contained in an object
    It returns an iterable of the solutions to the exact cover problem.
    """
    if not X:
        yield list(solution)
    else:
        c = min(X, key=lambda c: len(X[c]))
        for r in list(X[c]):
            solution.append(r)
            cols = select(X, Y, r)
            for s in solve(X, Y, solution):
                yield s
            deselect(X, Y, r, cols)
            solution.pop()
            
def select(X, Y, r):
        cols = []
        for j in Y[r]:
            for i in X[j]:
                for k in Y[i]:
                    if k != j:
                        X[k].remove(i)
            cols.append(X.pop(j))
        return cols
def deselect(X, Y, r, cols):
        for j in reversed(Y[r]):
            X[j] = cols.pop()
            for i in X[j]:
                for k in Y[i]:
                    if k != j:
                        X[k].add(i)

In [2]:
# Wiki example

I = {1, 2, 3, 4, 5, 6, 7}
Y = {
    'A': [1, 4, 7],
    'B': [1, 4],
    'C': [4, 5, 7],
    'D': [3, 5, 6],
    'E': [2, 3, 6, 7],
    'F': [2, 7]}

X = {j: set() for j in I}
for i in Y:
    for j in Y[i]:
        X[j].add(i)

next(solve(X, Y, solution=[]))

['B', 'D', 'F']

In [3]:
# Applied to rectangle positions

I = {'a','d',1,2,3,4}
Y = {'A0':['a',1,2],
     'A1':['a',1,3],
     'A2':['a',3,4],
     'A3':['a',2,4],
     'D1':['d',1,2]
    }

X = {j: set() for j in I}
for i in Y:
    for j in Y[i]:
        X[j].add(i)
        
next(solve(X, Y, solution=[]))

['D1', 'A2']

In [4]:
# https://stackoverflow.com/questions/492519/timeout-on-a-function-call

from __future__ import print_function
import sys
import threading
from time import sleep

try:
    import thread
except ImportError:
    import _thread as thread

def quit_function(fn_name):
    thread.interrupt_main() # raises KeyboardInterrupt
    
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

@exit_after(5)
def countdown(n):
    print('countdown started', flush=True)
    for i in range(n, -1, -1):
        sleep(1)
        print(i, end=', ', flush=True)

    print('countdown finished')
 
countdown(1)
try:
    countdown(4)
except:
    print('countdown interrupted')

countdown started
1, 0, countdown finished
countdown started
4, 3, 2, 1, countdown interrupted


In [5]:
# Could be generalized to an exact cover problem 
# for objects in any object rather than objects in a square

from collections import defaultdict
from itertools import product

def positions_in_square(objects, square_size):
    """
    Translates an iterable of objects (pentonimos, rectangles, ...)
    to all valid posistions in a square.
    
    The objects are an iterable of tiles,
    each described by a coordinate tuple. 
    It is assumed a (0,0) coordinate is included for each object.
    
    Projects coordinates to integers.
    """
    region_set = set(product(range(square_size), repeat=2))
    max_ = square_size**2
    region = list(region_set)
    positions = defaultdict(set)
    for ix, tiles in enumerate(objects):
        
        for (oi, oj), ri, rj, rk in product(region, (-1, +1), (-1, +1), (0, 1)):
            # Place origin of piece at oi, oj, reflecting vertically
            # if ri == -1, horizontally if rj == -1, and diagonally if
            # rk == 1.
            placing = []
            for t in tiles:
                p = ri * t[rk] + oi, rj * t[1-rk] + oj
                if p not in region_set:
                    break
                placing.append(p[0]*square_size+p[1])
            else:
                placing = tuple(sorted(placing, reverse=True))
                positions[ix].add(placing)
                
    return positions

def positions_to_constraints(positions):
    """
    Takes a dict of positions of objects
    Returns the positions as cover constraints, 
    where the same objects can not coexist with an alternate position.
    """
    nodes = defaultdict(set)
    objects = defaultdict(tuple)
    
    c = 0
    for x,y in positions.items():
        for z in y:
            c += 1
            for i in z:
                nodes[i].add(c)
            nodes[-x-1].add(c)
            objects[c] = (-x-1,)+z
        # allow not-used objects
        c += 1
        nodes[-x-1].add(c)
        objects[c] = (-x-1,)
    return nodes, objects

@exit_after(10)
def solution(objects,square_size):
    """
    Takes a list of objects and a square size, 
    and retuns an exact cover if there eixsts one.
    """
    p = positions_in_square(objects,square_size)
    X,Y = positions_to_constraints(p)
    try: 
        return [[divmod(x,square_size) for x in Y[y][1:]] 
                for y in next(solve(X,Y, solution=[]))]
    except: 
        return False

In [6]:
objects = [((0,-1),(0,0)),
           ((0,0),(0,1),(1,0),(1,1)),
           ((0,0),(0,1),(1,0),(1,1)),
           ((0,0),(0,1),(0,2)),]

square_size = 3

%timeit -n1 -r1 print(solution(objects,square_size))

[[(2, 2), (2, 1), (1, 2), (1, 1)], [], [(2, 0), (1, 0)], [(0, 2), (0, 1), (0, 0)]]
1 loop, best of 1: 7.83 ms per loop


In [7]:
# Rectangles only have 2 symmetries, rather than 8 for a random 2d object.

def positions_in_square(objects, square_size):
    """
    Translates an iterable of rectangles
    to all valid posistions in a square.
    
    The rectangles are an iterable of tiles,
    each described by a coordinate tuple. 
    It is assumed a (0,0) coordinate is included for each rectangle.
    
    Projects coordinates to integers.
    """
    region_set = set(product(range(square_size), repeat=2))
    max_ = square_size**2
    region = list(region_set)
    positions = defaultdict(set)
    for ix, tiles in enumerate(objects):
        for (oi, oj), rk in product(region, (0, 1)):
            # Reflect diagonally if rk = 1
            placing = []
            for t in tiles:
                p = t[rk] + oi, t[1-rk] + oj
                if p not in region_set:
                    break
                placing.append(p[0]*square_size+p[1])
            else:
                placing = tuple(sorted(placing))
                positions[ix].add(placing)
                
    return positions

In [8]:
objects = [((0,-1),(0,0)),
           ((0,0),(0,1),(1,0),(1,1)),
           ((0,0),(0,1),(1,0),(1,1)),
           ((0,0),(0,1),(0,2)),]

square_size = 3

%timeit -n1 -r1 print(solution(objects,square_size))

[[(1, 1), (1, 2), (2, 1), (2, 2)], [], [(0, 1), (0, 2)], [(0, 0), (1, 0), (2, 0)]]
1 loop, best of 1: 2.1 ms per loop


In [9]:
from math import ceil
from itertools import combinations_with_replacement as cwr

def mulx(x):
    return x[0]*x[1]

def minx(x):
    return x[0]-x[1]

def upper_bound(n):
    mx = ceil(((n**2)/3)**0.5)
    return (mx,mx),(n,n-mx),(n-mx,mx)

def limits_bound(n):
    a,b,c = map(mulx,upper_bound(n))
    return max(a,b,c), min(a,b,c)

def rectangles(n):
    a,b = limits_bound(n)
    return [x for x in cwr(range(1,n+1),2) if 1 < mulx(x) <= a]

def rectangles_to_tiles(R):
    return [list(product(range(x),range(y))) for x,y in R]

def cover_cost(a_solution):
    c = [len(x) for x in a_solution if x]
    return max(c)-min(c)


In [10]:
limits_bound(6)

(16, 8)

In [11]:
def surface_ranges(n):   
    r = sorted(rectangles(n),key = lambda x:mulx(x))
    p = []
    for ix,_ in enumerate(r):
        for iy,_ in enumerate(r):
            v = r[ix:iy+1]
            if sum(mulx(x) for x in v) >= n**2:
                p.append(sorted(v,key = lambda x:mulx(x), reverse=True))
    return p

def run(n):
    p = surface_ranges(n)
    p = [(mulx(x[0])-mulx(x[-1]),x) for x in p]
    limit = minx(limits_bound(n))
    d = defaultdict(list)
    for x,y in p:
        if 0 < x <= limit:
            d[x].append(y)
    for x,y in d.items():
        y.sort(key=lambda x:(1/len(x),mulx(x[0])), reverse=True)
    return d
run(4)

defaultdict(list,
            {1: [[(3, 3), (2, 4)]],
             3: [[(3, 3), (2, 4), (2, 3)], [(2, 3), (1, 4), (2, 2), (1, 3)]],
             4: [[(2, 4), (2, 3), (2, 2)],
              [(2, 4), (2, 3), (1, 4), (2, 2)],
              [(2, 3), (1, 4), (2, 2), (1, 3), (1, 2)]],
             5: [[(3, 3), (2, 4), (2, 3), (2, 2)],
              [(3, 3), (2, 4), (2, 3), (1, 4), (2, 2)],
              [(2, 4), (2, 3), (1, 4), (2, 2), (1, 3)]],
             6: [[(3, 3), (2, 4), (2, 3), (1, 4), (2, 2), (1, 3)],
              [(2, 4), (2, 3), (1, 4), (2, 2), (1, 3), (1, 2)]]})

In [28]:
def mondriaan(n):
    a,b = limits_bound(n)
    limit = a-b
    ranges = run(n)
    limits = sorted([x for x in ranges.keys() if x <= limit])

    while limits:
        changed = False
        limit = limits.pop()
        for a_range in ranges[limit]:
            try:
                v = solution(rectangles_to_tiles(a_range),n)
            except:
                v = None
            if v:
                last_sol = cover_cost(v)
                while limits and limits[-1] >= last_sol:
                    limits.pop()
                changed = True
                break

    return n,last_sol

mondriaan(4)

(4, 4)

In [ ]:
for x in range(3,18):
    %timeit -n1 -r1 print(mondriaan(x))

(3, 2)
1 loop, best of 1: 8.76 ms per loop
(4, 4)
1 loop, best of 1: 57 ms per loop
(5, 4)
1 loop, best of 1: 9.8 ms per loop
(6, 5)
1 loop, best of 1: 289 ms per loop
(7, 5)
1 loop, best of 1: 410 ms per loop
(8, 6)
1 loop, best of 1: 6.02 s per loop
(9, 6)
1 loop, best of 1: 1.66 s per loop
(10, 8)
1 loop, best of 1: 1min 40s per loop
(11, 6)
1 loop, best of 1: 31.9 s per loop
(12, 7)
1 loop, best of 1: 1min 38s per loop


In [13]:
# for x in [4,8,10,20]:
#    %timeit -n1 -r1 print(mondriaan(x))